# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 07:46:24] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=30756, nccl_port=None, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=937688277, constrained_json_w

[2025-07-09 07:46:24] Inferred chat template from model path: llama-2


[2025-07-09 07:46:32] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 07:46:32] Init torch distributed begin.
[2025-07-09 07:46:32] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 07:46:33] Load weight begin. avail mem=78.50 GB


[2025-07-09 07:46:33] The weight of LmHead is not packed
[2025-07-09 07:46:34] Using model weights format ['*.safetensors']


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.24s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.24s/it]

[2025-07-09 07:46:39] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-07-09 07:46:39] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-09 07:46:39] Memory pool end. avail mem=53.92 GB


[2025-07-09 07:46:39] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.14 GB


[2025-07-09 07:46:40] INFO:     Started server process [3503966]
[2025-07-09 07:46:40] INFO:     Waiting for application startup.
[2025-07-09 07:46:40] INFO:     Application startup complete.
[2025-07-09 07:46:40] INFO:     Uvicorn running on http://127.0.0.1:30756 (Press CTRL+C to quit)


[2025-07-09 07:46:40] INFO:     127.0.0.1:57346 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 07:46:41] INFO:     127.0.0.1:57356 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 07:46:41] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:46:41.174636


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 07:47:31] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-09T07:47:31.210827


[2025-07-09 07:47:33] INFO:     127.0.0.1:57360 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 07:47:33] The server is fired up and ready to roll!


[2025-07-09 07:47:36] INFO:     127.0.0.1:57372 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-09 07:47:36] Child process unexpectedly failed with exitcode=9. pid=3504182
[2025-07-09 07:47:36] Child process unexpectedly failed with exitcode=9. pid=3504115


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 07:47:42] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=32168, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=429656783, constrained_json_whi

[2025-07-09 07:47:43] Inferred chat template from model path: llama-2


[2025-07-09 07:47:50] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 07:47:50] Init torch distributed begin.


[2025-07-09 07:47:50] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 07:47:52] Load weight begin. avail mem=78.50 GB
[2025-07-09 07:47:52] The weight of LmHead is not packed


...

[2025-07-09 07:47:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.10 GB


[2025-07-09 07:47:58] INFO:     Started server process [3505301]
[2025-07-09 07:47:58] INFO:     Waiting for application startup.
[2025-07-09 07:47:58] INFO:     Application startup complete.
[2025-07-09 07:47:58] INFO:     Uvicorn running on http://127.0.0.1:32168 (Press CTRL+C to quit)


[2025-07-09 07:47:59] INFO:     127.0.0.1:46638 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 07:47:59] INFO:     127.0.0.1:46654 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 07:47:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:47:59.399680


[2025-07-09 07:48:02] INFO:     127.0.0.1:46666 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 07:48:02] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 07:48:04] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:48:04.221299


[2025-07-09 07:48:04] INFO:     127.0.0.1:46676 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-09 07:48:04] Child process unexpectedly failed with exitcode=9. pid=3505517


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 07:48:11] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=38540, nccl_port=None, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=846943058,

[2025-07-09 07:48:19] Casting torch.bfloat16 to torch.float16.


[2025-07-09 07:48:20] Casting torch.bfloat16 to torch.float16.
[2025-07-09 07:48:20] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 07:48:20] Init torch distributed begin.


[2025-07-09 07:48:20] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 07:48:21] Load weight begin. avail mem=78.50 GB


...

[2025-07-09 07:48:39] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-07-09 07:48:40] INFO:     Started server process [3506328]
[2025-07-09 07:48:40] INFO:     Waiting for application startup.
[2025-07-09 07:48:40] INFO:     Application startup complete.
[2025-07-09 07:48:40] INFO:     Uvicorn running on http://127.0.0.1:38540 (Press CTRL+C to quit)


[2025-07-09 07:48:40] INFO:     127.0.0.1:39542 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 07:48:41] INFO:     127.0.0.1:39556 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 07:48:41] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:48:41.262602


[2025-07-09 07:48:44] INFO:     127.0.0.1:39570 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 07:48:44] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 07:48:45] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:48:45.668573


[2025-07-09 07:48:45] INFO:     127.0.0.1:39586 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-07-09 07:48:45] Child process unexpectedly failed with exitcode=9. pid=3506543


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 07:48:53] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=32007, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=215048445, constrai

[2025-07-09 07:49:03] Casting torch.bfloat16 to torch.float16.


[2025-07-09 07:49:03] Casting torch.bfloat16 to torch.float16.
[2025-07-09 07:49:03] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 07:49:03] Init torch distributed begin.


[2025-07-09 07:49:04] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 07:49:05] Load weight begin. avail mem=78.50 GB


...

[2025-07-09 07:49:21] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-09 07:49:22] INFO:     Started server process [3507356]
[2025-07-09 07:49:22] INFO:     Waiting for application startup.
[2025-07-09 07:49:22] INFO:     Application startup complete.
[2025-07-09 07:49:22] INFO:     Uvicorn running on http://127.0.0.1:32007 (Press CTRL+C to quit)


[2025-07-09 07:49:23] INFO:     127.0.0.1:44396 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 07:49:23] INFO:     127.0.0.1:44402 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 07:49:23] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:49:23.568061


[2025-07-09 07:49:26] INFO:     127.0.0.1:44404 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 07:49:26] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 07:49:28] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:49:28.162739


[2025-07-09 07:49:28] INFO:     127.0.0.1:44418 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-09 07:49:28] Child process unexpectedly failed with exitcode=9. pid=3507571
[2025-07-09 07:49:28] Child process unexpectedly failed with exitcode=9. pid=3507505


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 07:49:35] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=30169, nccl_port=None, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=137332707, constrained_json_whitespace_pattern=None, watch

[2025-07-09 07:49:44] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 07:49:44] Init torch distributed begin.


[2025-07-09 07:49:45] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 07:49:46] Load weight begin. avail mem=78.50 GB
[2025-07-09 07:49:46] The weight of LmHead is not packed


[2025-07-09 07:49:46] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-09 07:49:51] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-09 07:49:51] INFO:     Started server process [3508383]
[2025-07-09 07:49:51] INFO:     Waiting for application startup.
[2025-07-09 07:49:51] INFO:     Application startup complete.
[2025-07-09 07:49:51] INFO:     Uvicorn running on http://0.0.0.0:30169 (Press CTRL+C to quit)


[2025-07-09 07:49:52] INFO:     127.0.0.1:39374 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 07:49:52] INFO:     127.0.0.1:39386 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 07:49:53] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:49:53.023025


[2025-07-09 07:49:54] INFO:     127.0.0.1:39394 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 07:49:54] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-09 07:49:57] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T07:49:57.589832


[2025-07-09 07:49:58] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: False, gen throughput (token/s): 10.58, #queue-req: 0, timestamp: 2025-07-09T07:49:58.193878


[2025-07-09 07:49:58] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, accept len: 1.80, cuda graph: False, gen throughput (token/s): 111.77, #queue-req: 0, timestamp: 2025-07-09T07:49:58.838079


[2025-07-09 07:49:59] Decode batch. #running-req: 1, #token: 234, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 107.74, #queue-req: 0, timestamp: 2025-07-09T07:49:59.487771


[2025-07-09 07:50:00] Decode batch. #running-req: 1, #token: 302, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 105.53, #queue-req: 0, timestamp: 2025-07-09T07:50:00.132149


[2025-07-09 07:50:00] Decode batch. #running-req: 1, #token: 370, token usage: 0.02, accept len: 1.70, cuda graph: False, gen throughput (token/s): 105.11, #queue-req: 0, timestamp: 2025-07-09T07:50:00.779104


[2025-07-09 07:50:01] Decode batch. #running-req: 1, #token: 436, token usage: 0.02, accept len: 1.65, cuda graph: False, gen throughput (token/s): 103.42, #queue-req: 0, timestamp: 2025-07-09T07:50:01.417248


[2025-07-09 07:50:01] INFO:     127.0.0.1:39408 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-09 07:50:01] Child process unexpectedly failed with exitcode=9. pid=3508598


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).